In [1]:
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import random
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process.kernels import ConstantKernel as C, Product
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.gaussian_process.kernels import Matern
from sklearn.utils.optimize import _check_optimize_result
import warnings
from sklearn.gaussian_process.kernels import RationalQuadratic


In [2]:
#不行就自己写高斯过程吧    
    #R=np.linalg.pinv(np.cov(Gaussian_train))
    #I=np.ones((trainset_size,1))
    
    #Y1, variance=gaussian.predict(U,return_std=True)
    #Y2, cov=gaussian.predict(U,return_cov=True)
    #print(Y1.shape,variance.shape,cov.shape) (50,1),(50,1),(50,50),第一个维度是原来的个体（行），第二个维度是新预测的个体
    #s_sqr=np.ones([best_pick_number,1])*100
    
    #for m in range(trainset_size):
    #    cov_value=np.expand_dims(cov[m,:],axis=0)
    #    compo=1-np.dot(np.dot(cov_value,R),cov_value.T)
    #    conpo=1/np.dot(np.dot(I.T,R),I)
    #    s_sqr[m]=np.sqrt(variance[m]**2*(compo+compo**2*conpo))

In [3]:

'''
该文件目的是：
1.接收到一组变量范围numpy矩阵以及样本需求个数，shape = (m,2)，输出样本numpy矩阵
执行ParameterArray函数即可
'''

def Partition (number_of_sample,
               limit_array):
    """
    为各变量的变量区间按样本数量进行划分，返回划分后的各变量区间矩阵
    :param number_of_sample: 需要输出的 样本数量
    :param limit_array: 所有变量范围组成的矩阵,为(m, 2)矩阵，m为变量个数，2代表上限和下限
    :return: 返回划分后的个变量区间矩阵（三维矩阵），三维矩阵每层对应于1个变量
    """
    coefficient_lower = np.zeros((number_of_sample, 2))
    coefficient_upper = np.zeros((number_of_sample, 2))
    for i in range(number_of_sample):
        coefficient_lower[i, 0] = 1 - i / number_of_sample
        coefficient_lower[i, 1] = i / number_of_sample
    for i in range(number_of_sample):
        coefficient_upper[i, 0] = 1-(i+1) / number_of_sample
        coefficient_upper[i, 1] = (i+1) / number_of_sample

    partition_lower = coefficient_lower @ limit_array.T  #变量区间下限
    partition_upper = coefficient_upper @ limit_array.T  # 变量区间上限

    partition_range = np.dstack((partition_lower.T, partition_upper.T))  # 得到各变量的区间划分，三维矩阵每层对应于1个变量
    return partition_range #返回区间划分上下限

def Representative(partition_range):
    """
    计算单个随机代表数的函数
    :param partition_range: 一个shape为 (m,N,2) 的三维矩阵，m为变量个数、n为样本个数、2代表区间上下限的两列
    :return: 返回由各变量分区后区间随机代表数组成的矩阵，每列代表一个变量
    """
    number_of_value = partition_range.shape[0]  #获得变量个数
    numbers_of_row = partition_range.shape[1]  # 获得区间/分层个数
    coefficient_random = np.zeros((number_of_value,numbers_of_row, 2))  # 创建随机系数矩阵
    representative_random = np.zeros((numbers_of_row, number_of_value))

    for m in range(number_of_value):
        for i in range(numbers_of_row):
            y = random.random()
            coefficient_random[m,i, 0] = 1 - y
            coefficient_random[m,i, 1] = y

    temp_arr = partition_range * coefficient_random  # 利用*乘实现公式计算（对应位置进行乘积计算），计算结果保存于临时矩阵 temp_arr 中
    for j in range(number_of_value): #计算每个变量各区间内的随机代表数，行数为样本个数n，列数为变量个数m
        temp_random = temp_arr[j, :, 0] + temp_arr[j, :, 1]
        representative_random[:,j] = temp_random
    return representative_random  # 返回代表数向量

def Rearrange(arr_random):
    """
    打乱矩阵各列内的数据
    :param arr_random: 一个N行, m列的矩阵
    :return: 每列打乱后的矩阵
    """
    for i in range(arr_random.shape[1]):
        np.random.shuffle(arr_random[:, i])
    return arr_random



def ParameterArray(limitArray,
                   sampleNumber):
    """
    根据输入的各变量的范围矩阵以及希望得到的样本数量，输出样本参数矩阵
    :param limitArray:变量上下限矩阵，shape为(m,2),m为变量个数
    :param sampleNumber:希望输出的 样本数量
    :return:样本参数矩阵
    """
    arr = Partition(sampleNumber, limitArray)
    parametersMatrix = Rearrange(Representative(arr))
    return  parametersMatrix


'''以下为类创建

class DoE(object):
    def __init__(self, name_value, bounds):
        self.name = name_value
        self.bounds = bounds
        self.type = "DoE"
        self.result = None


class DoE_LHS(DoE):
    # 拉丁超立方试验样本生成
    def __init__(self, name_value, bounds, N):
        DoE.__init__(self, name_value, bounds)
        self.type = "LHS"
        self.ParameterArray = ParameterArray(bounds, N)
        self.N = N

    def write_to_csv(self):
        """
        将样本数据写入LHS.csv文件，文件保存至运行文件夹内
        """
        sample_data = pd.DataFrame(self.ParameterArray, columns=self.name)
        sample_data.to_csv("LHS.csv")
'''

'以下为类创建\n\nclass DoE(object):\n    def __init__(self, name_value, bounds):\n        self.name = name_value\n        self.bounds = bounds\n        self.type = "DoE"\n        self.result = None\n\n\nclass DoE_LHS(DoE):\n    # 拉丁超立方试验样本生成\n    def __init__(self, name_value, bounds, N):\n        DoE.__init__(self, name_value, bounds)\n        self.type = "LHS"\n        self.ParameterArray = ParameterArray(bounds, N)\n        self.N = N\n\n    def write_to_csv(self):\n        """\n        将样本数据写入LHS.csv文件，文件保存至运行文件夹内\n        """\n        sample_data = pd.DataFrame(self.ParameterArray, columns=self.name)\n        sample_data.to_csv("LHS.csv")\n'

In [4]:
#拉丁超立方抽样 调用ParameterArray，传入bounds（先lb后ub，shape = (m,2)，m是变量的维度数，2是lb和ub）和N（样本数）
#输出为numpy的矩阵，shape为(N,m)

'''以下为使用

arr_limit = np.array([[-100, -100, -100, -1000, -1000, -1000, 0, 32, 8, 100],
                      [100, 100, 100, 1000, 1000, 100, 10, 2000, 100, 500]]).T
name_value = ["Fx", "Fy", "Fz", "Mx", "My", "Mz", "Pressure", "R", "nozzle_th", "nozzle_h"]  # 变量名

a=ParameterArray(arr_limit,20)
a.shape'''

'以下为使用\n\narr_limit = np.array([[-100, -100, -100, -1000, -1000, -1000, 0, 32, 8, 100],\n                      [100, 100, 100, 1000, 1000, 100, 10, 2000, 100, 500]]).T\nname_value = ["Fx", "Fy", "Fz", "Mx", "My", "Mz", "Pressure", "R", "nozzle_th", "nozzle_h"]  # 变量名\n\na=ParameterArray(arr_limit,20)\na.shape'

In [5]:
# ackley multimodal function
from numpy import arange
from numpy import exp
from numpy import sqrt
from numpy import cos
from numpy import e
from numpy import pi
from numpy import meshgrid
from matplotlib import pyplot
from mpl_toolkits.mplot3d import Axes3D

# objective function
def objective(X):
    return -20.0 * exp(-0.2 * sqrt((1/dimension) * np.sum(X**2)))- \
                       exp((1/dimension) * np.sum(cos(2 * pi * X))) + e + 20

In [45]:
#From the paper and requirements
dimension=10
initialization_population_size=5*dimension
best_pick_number=50
iteration=500 #需要改
F=0.8
CR=0.8
Omega=2
kernel1 = RationalQuadratic(length_scale=1.0, alpha=1.5,length_scale_bounds=(0,1e2))
kernel2 = RationalQuadratic(length_scale=1.0, alpha=1.5)+10
kernel3 = RationalQuadratic(length_scale=1.0, alpha=1.5)+100
kernel4 = RationalQuadratic(length_scale=1.0, alpha=1.5)+1000
kernel5 = RationalQuadratic(length_scale=1.0, alpha=1.5)+10000
kernel_list=[kernel1,kernel2,kernel3,kernel4,kernel5]

ub=np.ones([dimension,1])*10
lb=np.ones([dimension,1])*(-10)
bound=np.append(lb,ub,axis=1)

In [46]:
#这里只return了种群，如果需要适应度的话注意适应度的shape比较怪，argsort里面遇到过问题

def pick_best(Group_set,Fitness_set,pick_size):
    sorts_inceaseing=np.argsort(Fitness_set,axis=0)
    Best_pick=np.ones([pick_size,dimension])*100
    for i in range(pick_size):
        Best_pick[i]=Group_set[sorts_inceaseing[i]]
        #print(Fitness_set[sorts_inceaseing[i]])
    return Best_pick

# Step 1: initialization

In [47]:
#(50,10),50个个体，10维
Best_solution_so_far=np.ones([best_pick_number,dimension])*100

Group_initialization=ParameterArray(bound,initialization_population_size)
Group_Fitness_initialization=np.zeros(initialization_population_size)
for i in range(initialization_population_size):
    Group_Fitness_initialization[i]= objective(Group_initialization[i,:])

Group_Fitness_initialization=np.expand_dims(Group_Fitness_initialization,axis=1)
#Groups_with_Fitness=np.append(Group_initialization,Group_Fitness_initialization,axis=1)

Group=Group_initialization
Group_Fitness=Group_Fitness_initialization

# Step 2 will be reflected in the main function

In [48]:
#%%capture --no-display
#DE/rand
"""     r = random.sample(range(best_pick_number-1), 3)   # 随机选三个除了 i 以外的数字，作为选出的三个个体的索引
        for k in range(3):      # 选择方法使用 sample 先从 [0, PS-1) 中不重复的随机选择三个数字
            if r[k]>=j:     # 再将三个数字中，超过当前个体 i 的数字进行 +1
                r[k] += 1       # 实现避免选择到第 i 个个体的效果
        V[j]=Population_best[r[0]]+F*(Population_best[r[1]]-Population_best[r[2]])"""
def main(kernel):
    #(50,10),50个个体，10维
    Best_solution_so_far=np.ones([best_pick_number,dimension])*100

    Group_initialization=ParameterArray(bound,initialization_population_size)
    Group_Fitness_initialization=np.zeros(initialization_population_size)
    for i in range(initialization_population_size):
        Group_Fitness_initialization[i]= objective(Group_initialization[i,:])

    Group_Fitness_initialization=np.expand_dims(Group_Fitness_initialization,axis=1)
    #Groups_with_Fitness=np.append(Group_initialization,Group_Fitness_initialization,axis=1)

    Group=Group_initialization
    Group_Fitness=Group_Fitness_initialization

    for i in range(iteration):
        # Step 3 Pick best lambda individuals and update the best so far
        Population_best=pick_best(Group,Group_Fitness,best_pick_number)

        #Step 4
        U=np.ones([best_pick_number,dimension])*100
        V=np.ones([best_pick_number,dimension])*100
        for j in range(best_pick_number):


            #DE/Best
            r = random.sample(range(best_pick_number-1), 2)   # 随机选三个除了 i 以外的数字，作为选出的三个个体的索引
            for k in range(2):      # 选择方法使用 sample 先从 [0, PS-1) 中不重复的随机选择三个数字
                if r[k]>=j:     # 再将三个数字中，超过当前个体 i 的数字进行 +1
                    r[k] += 1       # 实现避免选择到第 i 个个体的效果

            V[j]=Population_best[0]+F*(Population_best[r[0]]-Population_best[r[1]])

            #Crossover 生成下一代
            jrand = random.randint(1, dimension)
            for k in range(dimension):
                if jrand == k or random.random()<=CR:
                    U[j,k] = V[j,k]
                else:
                    U[j,k] = Population_best[j,k]
        U=np.clip(U,-10,10)
        #END of Step 4  u = np.array([(pop[i].x[j], v[j])[random.random()<CR or jrand==j] for j in range(D)])
        #print(U.shape)

        #Step 5 Gaussian Process with sklearn
        if Group.shape[0]>=100:
            Gaussian_train=Group[-100:,:]
            Gaussian_y=Group_Fitness[-100:,:]
            #trainset_size=100
        else:
            Gaussian_train=Group
            Gaussian_y=Group_Fitness
            #trainset_size=Group.shape[0]

        gaussian=GaussianProcessRegressor(kernel=kernel)
        gaussian.fit(Gaussian_train,Gaussian_y)
        Y, variance=gaussian.predict(U,return_std=True)
        variance=np.expand_dims(variance,axis=1)

        #print(Y[0],variance[0])

        Y=Y-Omega*variance
        top_1_individual=np.argmin(Y)
        Fitness=np.expand_dims(np.expand_dims(objective(U[top_1_individual,:]),axis=0),axis=0)
        #print(Fitness,Y[top_1_individual])
        #print([top_1_individual],np.average(Group_Fitness),Fitness)

        Group=np.concatenate((Group,np.expand_dims(U[top_1_individual,:],axis=0)),axis=0)
        Group_Fitness=np.concatenate((Group_Fitness,Fitness),axis=0)
    #print(np.average(Group_Fitness[:50,:]),np.average(Group_Fitness[-50:,:]),np.min(Group_Fitness))
    return(Group[np.argmin(Group_Fitness),:])

In [49]:
%%capture --no-display
performance={}
for ker in kernel_list:
    perform=[0]
    for i in range(10):
        perform.append(main(ker))
    performance["{}".format(ker)]=sum(perform)/10

In [50]:
for ker in kernel_list:
    print(ker,performance["{}".format(ker)],"\n",objective(performance["{}".format(ker)]),"\n")

RationalQuadratic(alpha=1.5, length_scale=1) [ 9.51749062e-02 -4.21913619e-05  5.65574336e-05 -7.71334672e-05
  9.52176641e-02  9.51484172e-02 -9.52119703e-02 -5.52417371e-05
 -5.58168444e-05 -9.52064245e-02] 
 0.4934330786870298 

RationalQuadratic(alpha=1.5, length_scale=1) + 3.16**2 [ 5.04536629e-05 -5.21149884e-05 -2.89543743e-05 -3.10528252e-05
 -5.68823023e-05  2.76877606e-05 -1.69302832e-06 -1.26224863e-05
  6.38559462e-05 -3.28652783e-05] 
 0.0001620071897079356 

RationalQuadratic(alpha=1.5, length_scale=1) + 10**2 [ 5.21095849e-05  1.52542737e-05  1.04105466e-04  2.97775261e-06
  1.92644312e-05 -4.71747869e-05 -3.73444097e-05 -7.93104609e-05
 -5.25681471e-05 -1.06229055e-05] 
 0.0002078067516855242 

RationalQuadratic(alpha=1.5, length_scale=1) + 31.6**2 [ 1.07185639e-05 -3.39806199e-06  5.62921241e-05 -1.16120813e-05
 -1.03803510e-06  4.19244470e-05 -2.47295242e-05 -3.68192608e-05
 -6.24079101e-05  5.97178940e-05] 
 0.00015300656769667853 

RationalQuadratic(alpha=1.5, lengt